In [1]:
import pandas as pd
import re

# Load the Excel file
file_path = 'unstructured_data_input_file.csv'  
medical_data = pd.read_csv(file_path)

# Define the function to parse and extract the fields
def parse_and_extract(text, subject_id):
    fields = {
        "Date of birth:": "",
        "Sex:": "",
        "Service:": "",
        "Allergies:": "",
        "Attending:":"",
        "Chief Complaint:": "",
        "Major surgical or invasive procedure:": "",
        "History of present illness:": "",
        "Past medical history:": "",
        "social history:": "",
        "family history:": "",
        "Physical exam:": "",
        "Pertinent results:": "",
        "Brief Hospital Course:": "",
        "Medications on Admission:": "",
        "Discharge Medications:": "",
        "Discharge Disposition:": "",
        "Discharge Diagnosis:": ""
    }

    # Combine all field names into a regex pattern for the next field
    next_field_pattern = "|".join(r"\s*" + re.escape(field) + r"\s*" for field in fields.keys())

    for field in fields:
        
        # Capture everything until the next field, and allow multiple matches
        pattern = rf"\s*{re.escape(field)}\s*[:\s]*(.*?)(?=\s*{next_field_pattern})"
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)


        if matches:
            # Append all matches for the field, each on a new line
            fields[field] = "\n".join(match.strip() for match in matches)
            print(f"Subject ID '{subject_id}': Field '{field}' extracted successfully: {fields[field][:100]}...")  # Print the first 100 characters for context
        else:
            print(f"Subject ID '{subject_id}': Field '{field}' not found or empty in the text.")

    # Include 'subject_id' in the returned fields
    fields['subject_id'] = subject_id
    return fields

# Function to check if all fields are present in the text
def check_all_fields_present(text):
    for field in [
        "Date of birth:", "Sex:", "Service:", "Allergies:", "Attending:", "Chief Complaint:", "Major surgical or invasive procedure:",
        "History of present illness:", "Past medical history:", "social history:", "family history:", "Physical exam:",
        "Pertinent results:", "Brief Hospital Course:", "Medications on Admission:", "Discharge Medications:", "Discharge Disposition:","Discharge Diagnosis:"
    ]:
        pattern = rf"\s*{re.escape(field)}\s*[:\s]*"
        if not re.search(pattern, text, re.IGNORECASE):
            return False
    return True

# Apply the function to filter rows
filtered_medical_data = medical_data[medical_data['TEXT'].apply(check_all_fields_present)]
structured_data = filtered_medical_data.apply(lambda row: parse_and_extract(row['TEXT'], row['SUBJECT_ID']), axis=1)

# Convert the structured data into a DataFrame
structured_df = pd.DataFrame(list(structured_data))

# Reorder columns to have 'subject_id' first
structured_df = structured_df[['subject_id', 'Date of birth:', 'Sex:', 'Service:', 'Allergies:', 'Attending:','Chief Complaint:', 'Major surgical or invasive procedure:', 'History of present illness:', 'Past medical history:', 'social history:', 'family history:', 'Physical exam:', 'Pertinent results:', 'Brief Hospital Course:', 'Medications on Admission:', 'Discharge Medications:', 'Discharge Disposition:','Discharge Diagnosis:']]
# Save the structured DataFrame to a new Excel file
output_path = 'structured_format_output_file.csv'
structured_df.to_csv(output_path, index=False)

print(f"Data has been parsed, filtered, and saved successfully to {output_path}!")


Subject ID '26880': Field 'Date of birth:' extracted successfully: [**2080-1-4**]...
Subject ID '26880': Field 'Sex:' extracted successfully: M...
Subject ID '26880': Field 'Service:' extracted successfully: MEDICINE...
Subject ID '26880': Field 'Allergies:' extracted successfully: Patient recorded as having No Known Allergies to Drugs...
Subject ID '26880': Field 'Attending:' extracted successfully: [**First Name3 (LF) 1828**]...
Subject ID '26880': Field 'Chief Complaint:' extracted successfully: Mr. [**Known lastname 1829**] was seen at [**Hospital1 18**] after a mechanical fall from
a height o...
Subject ID '26880': Field 'Major surgical or invasive procedure:' extracted successfully: 1. Anterior cervical osteotomy, C6-C7, with decompression and
excision of ossification of the poster...
Subject ID '26880': Field 'History of present illness:' extracted successfully: Mr. [**Known lastname 1829**] is a 82 year old male who had a slip and fall
of approximately 10 feet...
Subject ID '26